# Testing Functions

This notebook was created to check whether the implemented functions in this project work correctly.

In [ ]:
# The autoreload extension allows you to tweak the code in the imported modules
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## File: dataset_processing.py

### Scaling number of datapoints from signal- to target- frequency:

Classification Signal

In [4]:
from dataset_processing import *

classification_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
classification_frequency = 1/30
target_frequency = 1/20

print("Classification array: ", classification_array)
print("Classification array shape: ", classification_array.shape)

reshaped_array = scale_classification_signal(
        signal = classification_array, 
        signal_frequency = classification_frequency,
        target_frequency = target_frequency
        )

print("\nReshaped array: ", reshaped_array)
print("Reshaped array shape: ", reshaped_array.shape)

del reshaped_array, classification_array

Classification array:  [0 1 2 3 4 5 6 7 8 9]
Classification array shape:  (10,)

Reshaped array:  [0 1 1 2 3 3 4 5 5 6 7 7 8 9 9]
Reshaped array shape:  (15,)


Continuous Signal

In [7]:
from dataset_processing import *

continuous_array = np.array([0., 1, 2, 3, 4, 5, 6, 7, 8, 9])
continuous_frequency = 2
target_frequency = 5

print("Continuous array: ", continuous_array)
print("Continuous array shape: ", continuous_array.shape)

reshaped_array = interpolate_signal(
        signal = continuous_array, 
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

print("\nReshaped array: ", reshaped_array)
print("Reshaped array shape: ", reshaped_array.shape)

del reshaped_array, continuous_array

Continuous array:  [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Continuous array shape:  (10,)

Reshaped array:  [0.  0.4 0.8 1.2 1.6 2.  2.4 2.8 3.2 3.6 4.  4.4 4.8 5.2 5.6 6.  6.4 6.8
 7.2 7.6 8.  8.4 8.8 9.  9. ]
Reshaped array shape:  (25,)


### Splitting a signal which is too long for the neural network model

In [18]:
from dataset_processing import *
import numpy as np # type: ignore

# Create random signal
frequency = 4
length_signal_seconds = 12.1 * 3600
signal = np.random.rand(int(length_signal_seconds * frequency))

# Only important parameters here:
nn_signal_seconds = 10 * 3600
shift_length_seconds = 3600
absolute_shift_deviation_seconds = 1800

signals_from_splitting, shift_length = split_long_signal(
        signal = signal, 
        sampling_frequency = frequency,
        target_frequency = frequency,
        signal_type = "feature",
        nn_signal_duration_seconds = nn_signal_seconds,
        wanted_shift_length_seconds = shift_length_seconds,
        absolute_shift_deviation_seconds = absolute_shift_deviation_seconds
        )

print("Shift length:", shift_length)
print(f"Shift length: {shift_length / frequency} seconds")
print("Signal shape: ", signal.shape)
print(f"Datapoints in NN: {nn_signal_seconds * frequency}")
print("Signals from splitting shape: ", signals_from_splitting.shape)

del signals_from_splitting, signal, shift_length

Shift length: 15120
Shift length: 3780.0 seconds
Signal shape:  (174240,)
Datapoints in NN: 144000
Signals from splitting shape:  (3, 144000)


### Divide up a signal into overlapping windows

Classification Signal

Continuous Signal

In [19]:
from dataset_processing import *

signal = np.random.rand(36000)

signal_in_windows = signal_to_windows(
    signal = signal, # type: ignore
    datapoints_per_window = 120,
    window_overlap = 90,
    signal_type = "feature"
    )

print(f"Signal shape: {signal.shape}")
print(f"Signal in windows shape: {signal_in_windows.shape}")

Signal shape: (36000,)
Signal in windows shape: (1197, 120)


In [26]:
import h5py
import numpy as np
import random

shhs_dataset = h5py.File("../Training_Data/SHHS_dataset.h5", 'r')
patients = list(shhs_dataset['slp'].keys()) # type: ignore

random_patient = patients[np.random.randint(0, len(patients))]
print(f"Random patient: {random_patient}")

print(np.unique(shhs_dataset["slp"][random_patient][:])) # type: ignore

for key in ["slp", "rri"]:
    print(f"\nkey: {key}")

    patient_data = shhs_dataset[key][random_patient][:] # type: ignore
    print(f"Data shape: {patient_data.shape}") # type: ignore

    data_freq = shhs_dataset[key].attrs["freq"] # type: ignore
    print(f"Data frequency: {data_freq}")
    print(f"Inverse data frequency: {1/data_freq}") # type: ignore

    print(f"Data length: {patient_data.shape[0]/data_freq} s") # type: ignore

Random patient: 200651_1
[0 1 2 3 5]

key: slp
Data shape: (739,)
Data frequency: 0.03333333333333333
Inverse data frequency: 30.0
Data length: 22170.0 s

key: rri
Data shape: (88680,)
Data frequency: 4
Inverse data frequency: 0.25
Data length: 22170.0 s


In [ ]:
from dataset_processing import *

random_array = np.random.rand(36000)
reshaped_array = reshape_signal(
    signal = random_array, # type: ignore
    sampling_frequency = 1,
    target_frequency = 4, 
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90,
    signal_type = "feature",
    nn_signal_duration_seconds = 10*3600,
    )